# Project Aletheia

## Introduction

## Installations

In [36]:
! pip install pandas
! pip install numpy
! pip install plotly
! pip install matplotlib
! pip install gensim
! pip install pyLDAvis


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Imports

In [37]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "notebook"

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import pickle


In [38]:
input_file_name = "../data/apify_text_topics_sentiment_analysis_vader_v1.csv"
topic_model_name = "../final_models/apify_data/text/gensim_bigrams_v3/55_topics_250_passes_120_iterations.pkl"

In [211]:
color_discrete_map={'positive':'blue',
                    'negative':'red',
                    'neutral':'yellow'
                    }

## Reading Data

In [39]:
## read the data
data = pd.read_csv(input_file_name);
data.head()

,url,author,date,title,soft_title,description,text,day,month,year,...,topic_probability_1,topic_probability_2,topic_probability_3,topic_words_1,topic_words_2,topic_words_3,negative,neutral,positive,compound
0,https://www.foxnews.com/politics/biden-says-xi...,Greg Norman,2022-11-14 00:00:00+00:00,Biden says after Xi meeting he doesn’t believe...,Biden says after Xi meeting he doesn’t believe...,President Biden said following his meeting wit...,President Biden told reporters Monday followin...,14,11,2022,...,0.481921,0.302021,0.059743,"china,chinese,taiwan,united_state,threat,defen...","biden,president,white_house,administration,joe...","organization,letter,position,school_board,empl...",0.028,0.870,0.102,0.9756
1,https://www.foxnews.com/politics/gop-rep-calve...,Sophia Slacik,2022-11-14 00:00:00+00:00,GOP Rep. Calvert wins election in competitive ...,GOP Rep. Calvert wins election in competitive ...,"The race for California 41st House district, o...",The Associated Press projects that Rep. Ken C...,14,11,2022,...,0.397262,0.201822,0.176006,"democrats,republican,republicans,democrat,demo...","flight,town,fire,bar,visitor,hall,beer,trip,ro...","voter,poll,vote,ballot,georgia,republican,voti...",0.015,0.910,0.075,0.8176
2,https://www.foxnews.com/politics/pelosi-not-ev...,Haris Alic,2022-11-14 00:00:00+00:00,Pelosi 'not even thinking' about political fut...,Pelosi 'not even thinking' about political fut...,House Speaker Nancy Pelosi’s spokesman said th...,House Speaker Nancy Pelosi’s spokesman forcefu...,14,11,2022,...,0.229805,0.204087,0.188578,"democrats,republican,republicans,democrat,demo...","attack,pelosi,violence,depape,speaker,wing,thr...","senate,vote,democrats,congress,legislation,rep...",0.016,0.889,0.095,0.9680
3,https://www.foxnews.com/politics/arizona-gover...,Paul Steinhauser,2022-10-25 00:00:00+00:00,"Katie Hobbs defeats GOP challenger Kari Lake, ...",Arizona gov election: Katie Hobbs defeats GOP ...,Democratic Secretary of State Katie Hobbs has ...,The Fox News Decision Desk can project that De...,25,10,2022,...,0.552282,0.129222,0.062635,"candidate,race,senate,republican,campaign,demo...","trump,president,republican,capitol,donald_trum...","school,education,teacher,theory,district,kid,s...",0.066,0.819,0.115,0.8934
4,https://www.foxnews.com/us/idaho-quadruple-hom...,Paul Best,2022-11-14 00:00:00+00:00,"'Crime of passion,' 'burglary gone wrong' amon...",Idaho quadruple student homicide: 'Crime of pa...,Idaho police are trying to narrow down a motiv...,Four college students were killed around 3:00 ...,14,11,2022,...,0.326747,0.253755,0.078065,"police,officer,victim,suspect,murder,incident,...","crime,los_angele,death,safe,city,resident,viol...","kid,mother,daughter,friend,mom,girl,husband,ba...",0.131,0.824,0.044,-0.9939


In [40]:
## exploring the shape of the data
data.shape

(7760, 27)

In [41]:
## Lets look at the data types of the columns
# read character column

pd.DataFrame(data.dtypes, columns=['DataType'])


,DataType
url,object
author,object
date,object
title,object
soft_title,object
description,object
text,object
day,int64
month,int64
year,int64


* We don't need data cleaning as we have pre-processed and cleaned this data in previous stages. 

## Topic Model

* We have already done topic modeling, but we can use the model to print topic names and their probabilities. 

In [68]:
## load the topic model
with open(topic_model_name, 'rb') as f:
    lda_model = pickle.load(f)

In [69]:
lda_model

In [84]:
## lets convert toics into a dataframe

## get the topics with 10 words
topics = lda_model.show_topics(num_topics=-1 ,formatted=False)

topic_object = []
for topic in topics:
    topic_word_object = {}
    topic_word_object["topic_number"] = topic[0]
    topic_words = topic[1]
    topic_words_sorted = sorted(topic_words, key=lambda x: x[1], reverse=True)
    for idx,word in enumerate(topic_words_sorted):
        topic_word_object[f"word_{idx}"] = "{word[0]},{word[1]:.4f}".format(word=word)
    topic_object.append(topic_word_object)

## convert to dataframe
topic_df = pd.DataFrame(topic_object)
topic_df

,topic_number,word_0,word_1,word_2,word_3,word_4,word_5,word_6,word_7,word_8,word_9
0,0,"church,0.0577","god,0.0559","faith,0.0310","christian,0.0256","religious,0.0213","prayer,0.0208","catholic,0.0139","name,0.0128","vaccination,0.0126","bible,0.0121"
1,1,"gun,0.0592","hunter,0.0342","email,0.0256","record,0.0236","laptop,0.0220","stelter,0.0194","hunter biden,0.0193","biden,0.0191","document,0.0143","new_york,0.0141"
2,2,"lincoln_project,0.0342","governor,0.0262","campaign,0.0214","new_york,0.0184","cuomo,0.0182","allegation,0.0180","weaver,0.0158","hochul,0.0139","schmidt,0.0105","johnson,0.0094"
3,3,"kid,0.0373","mother,0.0355","daughter,0.0304","friend,0.0288","mom,0.0213","girl,0.0167","husband,0.0167","baby,0.0164","boy,0.0137","dad,0.0133"
4,4,"organization,0.0116","letter,0.0094","position,0.0094","school_board,0.0090","employee,0.0087","opportunity,0.0085","board,0.0083","request,0.0081","practice,0.0079","book,0.0073"
5,5,"biden,0.1611","president,0.1347","white_house,0.0375","administration,0.0333","joe biden,0.0157","reporter,0.0147","obama,0.0117","vice_president,0.0108","speech,0.0107","trump,0.0103"
6,6,"energy,0.0839","oil,0.0824","gas,0.0562","production,0.0255","pipeline,0.0154","coal,0.0153","coach,0.0143","fuel,0.0141","domestic,0.0132","football,0.0129"
7,7,"senate,0.0609","walker,0.0369","campaign,0.0356","georgia,0.0311","warnock,0.0271","senator,0.0260","sander,0.0252","nominee,0.0248","primary,0.0210","ad,0.0182"
8,8,"wedding,0.0324","yellowstone,0.0292","coffee,0.0207","dress,0.0207","ring,0.0186","jeopardy,0.0145","ranch,0.0143","hotel,0.0139","flore,0.0130","room,0.0115"
9,9,"afghanistan,0.0747","taliban,0.0546","afghan,0.0223","withdrawal,0.0216","troop,0.0199","force,0.0190","kabul,0.0181","attack,0.0141","airport,0.0110","richard,0.0108"


In [93]:
## lets create a function to get word and probability for given topic number
def get_topic_word_distribution(topic_number):
    topic_dict = topic_df[topic_df["topic_number"] == topic_number].iloc[0].to_dict()
    word_dist = {}
    for idx in range(10):
        topic_word_prob = topic_dict[f"word_{idx}"]
        topic_word = topic_word_prob.split(",")[0]
        topic_prob = topic_word_prob.split(",")[1]
        word_dist[topic_word] = topic_prob
    return word_dist
        

## function to return pie chart for given topic number
def get_topic_word_distribution_chart(topic_number):
    df = pd.DataFrame(get_topic_word_distribution(topic_number).items(), columns=['word', 'probability'])
    fig = px.pie(df, values='probability', names='word', title=f"Topic {topic_number} Word Distribution")
    return fig

In [96]:
topic_df.shape

(55, 11)

### Plotting Topic Word Distribution

In [97]:
## lets plot distribution for for all topics
for topic_number in range(topic_df.shape[0]):
    fig = get_topic_word_distribution_chart(topic_number)
    fig.show()

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



...

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



## Exploratory Data Analysis

### Distribution of Sentiments

* Lets try and understand how the sentiments are distributed throughout the dataset. 
* We'll use `compound` column which holds compound sentiment based on `negative`, `neutral` and `positive` sentiment scores. 
* For that we'll need to feature engineer a new column to hold the value. 

In [42]:
## feature engineering - adding a new column for the sentiment
data["sentiment"] = data["compound"].apply(lambda x: "positive" if x > 0 else "negative" if x < 0 else "neutral")

In [43]:
## verify the new column
data.head()["sentiment"]

0    positive
1    positive
2    positive
3    positive
4    negative
Name: sentiment, dtype: object

In [44]:
## lets look at the summary of the data
data["sentiment"].value_counts()

positive    4331
negative    3421
neutral        8
Name: sentiment, dtype: int64

In [212]:
## lets create a pie chart to see the distribution of the sentiments
df = data["sentiment"].value_counts().rename_axis('sentiments').reset_index(name='counts')
fig = px.pie(df, values='counts',names='sentiments', title='Sentiment Distribution', color_discrete_map=color_discrete_map , color='sentiments')
fig.show()

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



### Topic Wise Distribution of Sentiments

* So we have top-3 topic details in our dataset. Topic 1,2,3 in order of their decreasing probability. 
* For topic wise distribution, lets start with topic numbers and see how the topics are distributed.  We can then engineer a way to look at topic words. 

In [98]:
data["topic_number_1"].value_counts()

30    427
53    357
24    317
46    306
49    289
41    258
3     253
5     241
45    234
36    226
23    219
35    219
33    214
37    213
39    201
28    186
4     182
21    181
25    173
51    169
52    154
27    149
32    148
42    142
12    139
40    135
43    129
29    128
26    127
2     123
11    106
9      99
10     98
15     87
1      86
34     84
20     80
7      74
44     65
50     64
38     63
0      60
18     58
22     53
13     50
31     50
47     50
14     48
16     47
48     46
6      45
17     34
8      32
19     27
54     15
Name: topic_number_1, dtype: int64

In [46]:
len(data["topic_number_1"].value_counts())

55

* So looks like we have atleast one label for each topic. 
* For now lets look at top 10 topics

In [102]:
top_10_topics = pd.DataFrame(data["topic_number_1"].value_counts().head(10).sort_values(ascending=False)).reset_index().rename(columns={"index":"topic_number","topic_number_1":"count"})
fig = px.bar(top_10_topics, x='topic_number', y='count')
fig.show()

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



#### News on Entertainment Industry

In [103]:
## lets checkout hightest topic
get_topic_word_distribution_chart(30)

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



* This was surprise number one, I kind of assumed that most common topic would be something related to `politics` or `global current events` did not expect a topic related to `Entertainment Industry`
* Lets check the news headlines to make sure we are mapping the topics correctly. 

In [105]:
## lets print topic 30 sample data
data[data["topic_number_1"] == 30].sample(10)["title"].values


array(['Cher gets a kiss during date night with new boyfriend, Alexander Edwards, who is 40 years her junior',
       "Sofia Vergara heats up Instagram with throwback bikini photo from the '90s",
       'Garth Brooks to headline drive-in concert events across America amid the coronavirus pandemic',
       'Jana Kramer, Mike Caussin finalize divorce',
       'Olivia Wilde dons high-waisted bikini on yacht outing with Harry Styles',
       "Joan Jett recalls enduring sexism as a young artist in the '70s",
       'Liam Payne and longtime girlfriend Cheryl announce separation',
       'Olivia Wilde addresses rumor Harry Styles spit on Chris Pine in Venice',
       "Quentin Tarantino begs Cannes audiences not to spoil 'Once Upon a Time in Hollywood'",
       "Elon Musk, Grimes' newborn’'s name, X Æ A-12, could hold up if challenged in court, legal expert says"],
      dtype=object)

* Holy Guacamole! They are related to `Hollywood`. Lets check the compound sentiment related to these topics

In [213]:
df = data[data["topic_number_1"] == 30]["sentiment"].value_counts().rename_axis('sentiments').reset_index(name='counts')
fig = px.pie(df, values='counts',names='sentiments', title='Sentiment Distribution For Topic 30', color_discrete_map=color_discrete_map , color='sentiments')
fig.show()

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



* So most of them are positive, but there are few which are negative, lets explore them in a bit more detail.

In [121]:
## lets start with positive sentiment
data.loc[(data["topic_number_1"] == 30) & (data["sentiment"] == "positive"), ["title", "compound", "sentiment"]].sample(10).values

array([['Liza Minnelli recalls how Judy Garland helped her cope with stage fright',
        0.9948, 'positive'],
       ['Texas State University offering Harry Styles course', 0.8871,
        'positive'],
       ["Anne Hathaway to join Rihanna, Mindy Kaling and Sandra Bullock in gender bending 'Ocean's Eleven' spinoff",
        0.9859, 'positive'],
       ["A look at the 'Goodfellas' star's life behind the camera and as a Hollywood legend",
        0.9973, 'positive'],
       ['Doris Day was not a recluse, spent her final years responding to fans, pal says',
        0.9997, 'positive'],
       ["Shia LaBeouf denies Olivia Wilde's claim that he was fired from 'Don’t Worry Darling'",
        0.9944, 'positive'],
       ['Actors Ben Savage, Anson Williams run for office, joining stars like Arnold Schwarzenegger in politics',
        0.994, 'positive'],
       ["'Crazy Rich Asians' draws immigrant parents to theaters, some for first time in years",
        0.9976, 'positive'],
       ["'Le

In [122]:
data.loc[(data["topic_number_1"] == 30) & (data["sentiment"] == "negative"), ["title", "compound", "sentiment"]].sample(10).values

array([["Mary Tyler Moore was 'devastated,’ plagued with guilt over her son’s tragic death, book claims",
        -0.981, 'negative'],
       ["Johnny Manziel says he's 'very sad' over split with Instagram model wife Bre Tiesi",
        -0.7089, 'negative'],
       ["Jennifer Lopez and Sean 'Diddy' Combs", -0.8627, 'negative'],
       ['Webb Space Telescope snaps spooky image of Pillars of Creation',
        -0.2052, 'negative'],
       ["'Don't Worry Darling' star Harry Styles appears to spit on co-star Chris Pine in viral video",
        -0.8049, 'negative'],
       ['Rising British actress Mya-Lecia Naylor dies suddenly at age 16',
        -0.1779, 'negative'],
       ['Drew Barrymore spills on Lucy Liu and Bill Murray’s ‘Charlie’s Angels’ on-set clash',
        -0.3379, 'negative'],
       ["David Miscavige's dad says wife of Scientology leader will 'never be free'",
        -0.9719, 'negative'],
       ["Lucy Liu recalls spat with Bill Murray on 'Charlie's Angels' set",
        -0

#### News in Crime/Policing

* Lets look at one more from top 10, because right now we are not interested in `Entertainment` news. :)

In [128]:
## looking at second highest topic
## lets checkout hightest topic
get_topic_word_distribution_chart(53)

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



* So this topics, looks related to `crime`, `law enforcements` and `shooting`. Lets look at some sample to confirm

In [129]:
## lets print topic 53 sample data
data[data["topic_number_1"] == 53].sample(10)["title"].values


array(["Fox News's Gianno Caldwell shares tragic news of his teenage brother's murder in Chicago",
       'Jacqueline Avant murder suspect arrested in Beverly Hills slaying',
       'Grieving mom of man killed, public safety experts weigh in amid spike in shootings',
       'Kylie Rae Harris crash a shock for NM fire chief; learns his 16-year-old daughter also died',
       'Mounted NYPD officer chases down robbery suspect in Times Square, video shows',
       "Do AOC's constituents back her calls to defund the police as violent crime surges in the district?",
       "I've never seen NYC crime as bad as it is right now",
       'Florida officer puts herself between speeding drunk driver and runners in 10K, dashcam video shows',
       'Austin City Council just voted to cut up to $150 million from its police department',
       'Body of missing Florida girl, 13, found; teen suspect in custody, sheriff says'],
      dtype=object)

* As expected they are not only related to various unfortunate `crimes`, but also related to articles related `police brutality`. 
* I think majority of these topics would be tagged `negative`. Lets explore the sentiments for this topic. 

In [214]:
df = data[data["topic_number_1"] == 53]["sentiment"].value_counts().rename_axis('sentiments').reset_index(name='counts')
fig = px.pie(df, values='counts',names='sentiments', title='Sentiment Distribution For Topic 53', color_discrete_map=color_discrete_map , color='sentiments')
fig.show()

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



* As expected majority of them are tagged as `negative`, but few of them are tagged positive. This could either be `sentiment analysis` error or `topic modeling` error. Lets verify it. 

In [131]:
## lets start with positive sentiment
data.loc[(data["topic_number_1"] == 53) & (data["sentiment"] == "positive"), ["title", "compound", "sentiment"]].sample(10).values

array([['12,000 fentanyl pills found packaged in candy at LAX TSA checkpoint',
        0.6486, 'positive'],
       ['Texas fire dog unlocks door after officials get locked out',
        0.9468, 'positive'],
       ['NJ teen dies after climbing into dumpster, getting crushed by recycling truck',
        0.6436, 'positive'],
       ["United flight attendant taken to hospital following incident with 'disruptive customer'",
        0.1686, 'positive'],
       ['Shia LaBeouf friends with police officer who arrested him in 2017',
        0.9458, 'positive'],
       ['Dutch police arrest suspect in thefts of Van Gogh, Frans Hals paintings',
        0.5719, 'positive'],
       ['San Francisco car break-ins up over 750% in some areas, police say',
        0.7269, 'positive'],
       ['New York City father attacked in front of 5-year-old son in broad daylight, video shows',
        0.9433, 'positive'],
       ['Texas authorities rescue 70 missing children victimized by sex trafficking',
        

* So some of these stories are positive, e.g.
    * `Texas fire dog unlocks door after officials get locked out` does have a fun positive sound to it. 
    * `Shia LaBeouf friends with police officer who arrested him in 2017` also this seems to have a happy wholesome ending to it. 
* But there are few which do not sound positive, but if you see the `compound` score, its quite low, that means the story is overall negative but some `positive` sounding words might have tipped the scale. For e.g.
    * `United flight attendant taken to hospital following incident with 'disruptive customer'` has score of 0.168 which is just above 0. 
* Lets explore one story which high positive score, but it should be negative. 

In [139]:
data.loc[data["title"] == "San Francisco car break-ins up over 750% in some areas, police say", ["text", "negative", "positive", "neutral", "compound"]].values

array([['Reports of vehicle break-ins in San Francisco were up by between 100% and over 750% in parts of the Golden City, according to the police department’s most recent monthly statistics.\n\nThe number of thefts from vehicles reported citywide in the month of May was more than double, from 923 in 2020 to 1891 the same month in 2021, San Francisco Police Department () data shows. The department has not yet released statistics for the month of June.\n\n closer examination of more specific locations within San Francisco show some police stations reported even bigger jumps, such as the Central Station, which reported a 753% increase in thefts from vehicles in May 2021 – with 725 – compared to the 85 reported during the same month in 2020, police department statistics show.\n\n’s Park Station saw a 110% increase year over year, with 48 in May 2020 and 101 in May 2021, and Richmond Station recorded a 102% uptick, from 100 to 202 incidents reported, records show.\n\nLocal news station  5 s

* Its hard to tell from the text why its tagged `positive` sentiment, I think looking at and extending `vader's` vocabulary might help us resolve this.  

### Comparing Political News

* My intuition tells me that if there is any opinion and bais it could be in political topics
* So lets explore 2 topics which are on opposite side of politics. 
* Topic Number 5 - Primarily consists of `words` and `pharases` related to `Democratic Party`
* Topic Number 36 - Primarily consists of `words` and `pharases` related to `Republican Party`

#### Exploring Sentiments For Democratic Party

* Lets jump to `Topic Number 5` which is related to politics. 

In [159]:
get_topic_word_distribution_chart(5)

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



* Lets look at some sample headlines for this topic. 

In [143]:
## lets print topic 5 sample data
data[data["topic_number_1"] == 5].sample(10)["title"].values

array(["MSNBC bringing back Jon Meacham shows 'there is no accountability' in media",
       'Karine Jean-Pierre blasted for response to record-border crossings',
       "MSNBC's Nicolle Wallace suggests Sen. Ron Johnson is 'in cahoots' with Russia to hurt Biden",
       'Washington Post column cheers Democrats ‘moving towards the attack,’ calls Biden’s hits on GOP ‘rousing’',
       "Biden has 'very low bar' compared to Trump press conferences",
       'Psaki says Biden intends to run in 2024 despite plummeting poll numbers',
       "WH fires back at journalists who call Biden speech 'political,' uses media's own reporting against them",
       'Biden ridiculed for complaining about reporters shouting questions',
       'Biden may have been forced into supporting student loan handout, Kilmeade suggests',
       'Biden uses quote notably uttered by Mao Zedong during big-money fundraiser'],
      dtype=object)

* Eye balling the headlines it seems that the topics assigned do summarize the news correctly. 
* So the topic majority words consistent of `Democratic` part and its presidents. Lets explore the sentiments for this topic

In [215]:
df = data[data["topic_number_1"] == 5]["sentiment"].value_counts().rename_axis('sentiments').reset_index(name='counts')
fig = px.pie(df, values='counts',names='sentiments', title='Sentiment Distribution For Topic 5', color_discrete_map=color_discrete_map , color='sentiments')
fig.show()

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



* Interestingly its almost a 50-50 divide in sentiments, and shockingly there are no `neutral` articles. So one thing we can safely deduce that this news agency is very opinionated on this topic. 
* Lets explore some sample articles on each sentiments.  

In [150]:
## lets start with positive sentiment
data.loc[(data["topic_number_1"] == 5) & (data["sentiment"] == "positive"), ["title", "negative", "positive", "compound", "sentiment"]].sample(10).values

array([['Obama offers meatless menu for his 60th birthday party', 0.039,
        0.088, 0.9349, 'positive'],
       ["Barack and Michelle Obama's $15 million house of hypocrisy",
        0.075, 0.147, 0.9917, 'positive'],
       ['Press secretary constantly has someone by her side at briefings',
        0.06, 0.093, 0.9812, 'positive'],
       ['Critics blast reporters as ‘afraid to really challenge’ Biden during presser',
        0.076, 0.09, 0.8352, 'positive'],
       ["'Vote like your life depends on it, because it does'", 0.094,
        0.126, 0.9548, 'positive'],
       ['Biden pick privately trashed him in 2015, said he would make Clinton look better by comparison',
        0.053, 0.107, 0.978, 'positive'],
       ["Biden slammed for hosting Inflation Reduction Act celebration with ‘American's 401Ks going down the drain’",
        0.053, 0.076, 0.9008, 'positive'],
       ["Nets' Kyrie Irving tells Biden to get Brittney Griner home",
        0.078, 0.168, 0.9929, 'positive'],
  

* Interestingly the most of the headlines that are tagged `positive` have a high score, all of them are in `90s`
* Lets look at some article details

In [152]:
data[data["title"] == "The president is batting nearly a thousand"]["text"].values[0]

'President Biden averaged nearly a gaffe per workday over the past four weeks, errors that included mispronunciations, confusion, inaccuracies, and forgetting that a congresswoman had died.\n\nBiden is hitting .810 from September 26 to October 24, committing at least 17 gaffes.\n\nThe president mostly fails to get on base over the long weekends when he often travels to his home state of Delaware and is generally kept away from microphones and cameras. One exception was his iconic bike fall in Rehoboth Beach, but that was back in June. He spent eight days in Delaware over the four-week stretch. It wouldn\'t be fair to count those days toward his average, since he usually doesn\'t get a chance at bat.\n\nBiden, as he wished a happy birthday to Vice President Kamala Harris Monday, called her a "great president." He later called Harris the "highest ranking black Indian, with Indian background, woman, in American history to be Vice President." That same day, Biden botched United Kingdom Pri

* I think in this article its quite obvious why its marked to have `positive` sentiment because quite a few high ranking positive words

In [154]:
## lets start with positive sentiment
data.loc[(data["topic_number_1"] == 5) & (data["sentiment"] == "negative"), ["title", "compound", "sentiment"]].sample(10).values

array([["White House says oil production 'not the focus' of conversations with Saudi Arabia",
        -0.934, 'negative'],
       ["McCarthy says Biden 'vilified' Americans, hits Dems on inflation, immigration and more in campaign speech",
        -0.5682, 'negative'],
       ["NBC's Lester Holt suggests his Kamala Harris interview sparked her decision to finally visit the border",
        -0.6067, 'negative'],
       ["Sanders hires adviser who blamed America for 9/11, questioned bin Laden op, mocked Michelle Obama's looks",
        -0.9844, 'negative'],
       ["White House won't say if Biden will bring up Jamal Khashoggi in Saudi meeting after widow breaks silence",
        -0.9851, 'negative'],
       ["White House defends US-Saudi 'strategic' partnership, after Biden vowed to make country a 'pariah' state",
        -0.4019, 'negative'],
       ["Media love Biden's high gas prices while ordinary Americans struggle with pump records",
        -0.9696, 'negative'],
       ['Sean Penn

* Lets look at a negative sentiment article as well. 

In [155]:

data[data["title"] == "Media love Biden's high gas prices while ordinary Americans struggle with pump records"]["text"].values[0]

'Gas prices continue to spike to record highs thanks to the efforts of the Biden administration. And no one could be happier about that than the media even though gas is nearly $5.00 a gallon.\n\nPresident Joe Biden admitted the high prices were intentional just recently. He told a press conference that "when it comes to the gas prices, we’re going through an incredible transition." As if that wasn’t clear enough, he added, "God willing, when it’s over, we’ll be stronger and the world will be stronger and less reliant on fossil fuels when this is over."\n\nLouisiana Republican Congressman Steve Scalise correctly responded, noting, "They’re saying the quiet part out loud now. They’re causing you pain at the pump because it’s all part of their radical agenda." There was almost no legacy media coverage of Biden’s radical statement or the response.\n\nThe national average of $4.86 a gallon is the daily reminder of that green agenda. And the news are ever eager to do  for Team Green. The Ne

* From the article language it does feel it has a strong opiniated `negative` sentiment. 

#### Exploring Sentiments for Republican Party

* Lets start by exploring topic words

In [160]:
get_topic_word_distribution_chart(36)

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [164]:
## lets print topic 5 sample data
data[data["topic_number_1"] == 36].sample(10)["title"].values

array(["McCarthy points to Raskin, Schiff ties to impeachments as evidence of 'obsession'",
       'Last pro-impeachment Republican faces almost certain defeat. How did the others fare in GOP primaries?',
       "Cheney defiant after House GOP vote, says she'll do 'everything I can' to keep Trump from Oval Office",
       "Collapsed Lafayette Square narrative is yet another 'fact-checked' story gone bad",
       "Trump announces 'endorsement' of impeachment manager, other New York Democrats in sarcastic posts",
       "Trump will 'probably' announce 2024 plans after midterms",
       'Republican Susan Collins, key swing-vote senator, announces she will vote to acquit Trump',
       "Trump's Mar-a-Lago stays busy as Republican candidates make fundraising pilgrimages",
       'Liz Cheney gets 2022 primary challenger after Trump impeachment vote',
       'Former Trump adviser Peter Navarro pleads not guilty to contempt of Congress charges'],
      dtype=object)

* Looks like most of them are tagged correctly in this sample, I did see a mistagged news in previous sample, but then couldn't find it again.  For now I'll assume its within error margin. 

In [216]:
df = data[data["topic_number_1"] == 36]["sentiment"].value_counts().rename_axis('sentiments').reset_index(name='counts')
fig = px.pie(df, values='counts',names='sentiments', title='Sentiment Distribution For Topic 36', color_discrete_map=color_discrete_map , color='sentiments')
fig.show()

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



* So we do see that `positive` vs `negative` sentiments are not evenly distributed, with heavy biasing towards `positive` news. 
* Lets explore headlines for both sentiments and see if we can get some more insights. 

In [166]:
## lets start with positive sentiment
data.loc[(data["topic_number_1"] == 36) & (data["sentiment"] == "positive"), ["title", "negative", "positive", "compound", "sentiment"]].sample(10).values

array([['Trump puts pressure on Pence ahead of election showdown in Congress',
        0.05, 0.09, 0.9902, 'positive'],
       ["Top Republicans flock to Reagan Library to speak out on GOP's future",
        0.018, 0.131, 0.9985, 'positive'],
       ["GOP megadonor Sheldon Adelson had 'massive impact' on Republican politics",
        0.048, 0.162, 0.999, 'positive'],
       ['How to wear sunscreen the right way', 0.04, 0.116, 0.9993,
        'positive'],
       ['Meet the potential GOP presidential contenders', 0.061, 0.095,
        0.997, 'positive'],
       ['Some conservatives turn on Trump for attacking Ron DeSantis ahead of midterms',
        0.026, 0.105, 0.9844, 'positive'],
       ['Secret Service official Tony Ornato retires after 25 years amid Jan. 6 probe',
        0.035, 0.065, 0.6597, 'positive'],
       ['Trump dominates 2024 GOP presidential nomination straw poll at Turning Point USA summit',
        0.032, 0.059, 0.9103, 'positive'],
       ['Rashida Tlaib calls Trump a

* We do see on mistagged news article over here, e.g. `How to wear sunscreen the right way`, this  could be that it didn't get tagged into any topic because of rarity and got mistagged here.  If we run topic modeling again it might be tagged into another topic. 
* Lets explore couple of headlines in detail to see if we agree with the tagged sentiment.

In [168]:
data[data["title"] == "Trump puts pressure on Pence ahead of election showdown in Congress"]["text"].values[0]

'President Trump is turning up the heat on Vice President Mike Pence on the eve of a joint session of Congress that is all but certain to certify President-elect Joe Biden’s Electoral College victory over Trump.\n\nPence, as vice president, will preside over the joint session, but is expected to play a mostly ceremonial role.\n\n      \'      \n\nBut Trump, in what may be his final attempt to try and reverse his presidential election defeat, is insisting that Pence has the power to overturn the election results.\n\n"The Vice President has the power to reject fraudulently chosen electors," the president tweeted on Tuesday morning.\n\nHours earlier, at an election eve rally in Georgia for the two  senators running in the state’s twin Senate runoff contests, Trump told the large crowd of supporters,\xa0" hope Mike Pence comes through for us,  have to tell you."\n\n" hope that our great vice president\xa0–\xa0our great vice president, comes through for us. He\'s a great guy. Of course, if 

* I think `positive` words like `Great`, `Like` and `Success`, `Support` etc might have an positive impact on the sentiment score for this news article. It doesn't look strongly positive to me, but it also doesn't look `negative` or `neutral`. 

In [167]:
data.loc[(data["topic_number_1"] == 36) & (data["sentiment"] == "negative"), ["title", "negative", "positive", "compound", "sentiment"]].sample(10).values

array([["GOP Sen. Ben Sasse says there is 'brokenness' in Trump's soul, refuses to rule out impeachment",
        0.081, 0.048, -0.9808, 'negative'],
       ['Biden says ‘poor kids’ just as bright as ‘white kids’ in latest gaffe',
        0.068, 0.071, -0.6249, 'negative'],
       ["Mother of Capitol Police officer Brian Sicknick blames 'people like Kari Lake' for his death",
        0.133, 0.115, -0.9038, 'negative'],
       ['Wyoming Republicans censure Cheney over Trump impeachment vote',
        0.117, 0.046, -0.9895, 'negative'],
       ['Top 5 moments from latest Jan. 6 hearing before midterm elections include Trump subpoena, new video',
        0.062, 0.042, -0.9753, 'negative'],
       ['Mike Lee, Evan McMullin spar over Jan. 6 riot', 0.117, 0.076,
        -0.9854, 'negative'],
       ["The Trump-Russia collusion hoax was a 'dirty trick' sanctioned by Hillary Clinton",
        0.087, 0.065, -0.3314, 'negative'],
       ['Who is Adam Kinzinger? Conservative congressman turned an

In [169]:
data[data["title"] == "GOP Sen. Ben Sasse says there is 'brokenness' in Trump's soul, refuses to rule out impeachment"]["text"].values[0]

' Sen. Ben Sasse said Friday he is willing to consider articles of impeachment against President Trump if they are introduced by the House of Representatives following a violent mob attack by pro-Trump supporters on the nation\'s Capitol earlier this week.\n\n .          \n\n"The House, if they come together and have a process,  will definitely consider whatever articles they might move," Sasse, -Neb., said on "\xa0This Morning."\n\nTop congressional leaders -- including House Speaker Nancy Pelosi, -Calif., and Democratic\xa0Leader Sen. Chuck Schumer -- are expected to hold a phone call on Friday to discuss ways to remove Trump from office. Among the options are persuading Vice President Mike Pence and the majority of the Cabinet to invoke the 25th Amendment, which would dub Trump unfit to serve and install Pence as the new . . leader, less than two weeks before President-elect Joe Biden is scheduled to be sworn in.\n\nPelosi has said if Pence does not act, she will move to impeach Tru

* In this case I think its obvious that words like `violent`, `attack` and `wild` are tipping the scale towards negative sentiment. 

### Analyzing Authors

* Lets explore how `Authors` are distributed and see if we can find any patterns in their writing styles

In [177]:
## lets look at the top 5 authors
data["author"].value_counts().sort_values(ascending=False).head(5)

Associated Press     260
Joseph Wulfsohn      204
Stephanie Nolasco    199
Charles Creitz       181
Paul Steinhauser     179
Name: author, dtype: int64

* Here some of the values are part of data clean up that we did in previous `notebooks`. 
* Any articles with `Fox News` author means, we weren't able to scrape the information. 
* Lets first see topic distribution for these authors. 

In [180]:
def get_topic_count_for_author(author_name):
    return pd.DataFrame(data[data["author"] == author_name]["topic_number_1"].value_counts().sort_values(ascending=False)).reset_index().rename(columns={"index":"topic_number","topic_number_1":"count"})


#### Associated Press

* Lets look at top 5 topics written by `Associated Press` 

In [219]:
ap_topics = get_topic_count_for_author("Associated Press")
fig = px.pie(ap_topics.head(5), values='count',names='topic_number', title='Topic Distribution For Associated Press')
fig.show()

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



* So majority was topic `46` and then topic `37`, lets see the topic words

In [187]:
get_topic_word_distribution_chart(46)

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [191]:
## lets checkout few headlines for topic 46 by Associated Press
data[(data["topic_number_1"] == 46) & (data["author"] == "Associated Press")].sample(10)["title"].values

array(['Wisconsin judge prohibits voters from canceling absentee ballots, casting new ones',
       "North Dakota Democrats hope to take back congressional seats from GOP. Here's what to expect on Tuesday.",
       'Michigan absentee ballot numbers indicate large voter turnout',
       'Arizona has several competitive House races, while Senate race could determine which party gains majority',
       'Noem, Smith headline South Dakota elections, while Republicans Thune, Johnson expected to return to Congress',
       "Virginia's election is highlighted by 3 congressional races in newly drawn districts",
       "How states can ensure that dead people's ballots are not counted",
       "Minnesota GOP secretary of state wants to mandate proofreading for every county's ballots",
       "Missouri judge halts parts of new voting law prior to Tuesday's election",
       'Michigan Supreme Court is urged to place proposed voting amendment on fall ballot'],
      dtype=object)

* From the headlines looks like, most of them are neutral or positive news articles or atleast the headlines are neutral sounding 
* Lets checkout the sentiment distribution for this. 

In [220]:
## sentiment distribution for topic 46 by Associated Press
ap_sentiments = data[(data["topic_number_1"] == 46) & (data["author"] == "Associated Press")]["sentiment"].value_counts().rename_axis('sentiments').reset_index(name='counts')
fig = px.pie(ap_sentiments, values='counts',names='sentiments', title='Sentiment Distribution For Topic 46 By Associated Press', color_discrete_map=color_discrete_map , color='sentiments')
fig.show()

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



* Most of them are `positive`, I was hoping for some facts reporting with `neutral` sentiment. 
* Lets checkout the `negative` sentiment headlines to get some insight. 

In [197]:
## lets checkout title for negative sentiment news for topic 46 by Associated Press
data.loc[(data["author"] == "Associated Press") & (data["topic_number_1"] == 46) & (data["sentiment"] == "negative"), ["title", "compound", "sentiment"]].values

array([['In Arizona, voting machine glitch gives way to election integrity concerns',
        -0.9731, 'negative'],
       ["Minnesota GOP secretary of state wants to mandate proofreading for every county's ballots",
        -0.929, 'negative'],
       ['Wisconsin judge prohibits voters from canceling absentee ballots, casting new ones',
        -0.9883, 'negative'],
       ['Michigan absentee ballot numbers indicate large voter turnout',
        -0.9011, 'negative'],
       ["How states can ensure that dead people's ballots are not counted",
        -0.9987, 'negative'],
       ['Wisconsin judge rejects attempt to allow absentee ballots with incomplete addresses to be counted',
        -0.875, 'negative'],
       ["Computer experts, election security experts warn of 'serious threat' to Georgia voting machines",
        -0.5927, 'negative'],
       ['Pennsylvania mail-in voting law challenged by Republicans',
        -0.9481, 'negative']], dtype=object)

* Reviewing these headlines and articles tells me, we would get more insight if we could figure out a way to compare this with absolute neutral vocabulary. Unfortunately we don't have that but definately a way to enhance this EDA. 
* Lets have a quick look at overall sentiment distribution for `AP`

In [222]:
## lets plot the sentiment distribution for all Associated Press news
ap_sentiments = data[(data["author"] == "Associated Press")]["sentiment"].value_counts().rename_axis('sentiments').reset_index(name='counts')
fig = px.pie(ap_sentiments, values='counts',names='sentiments', title='Sentiment Distribution For Topic 46 By Associated Press', color_discrete_map=color_discrete_map , color='sentiments')
fig.show()

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



* We can further break this down to see topic number and counts with sentiments. 

In [224]:
## lets checkout topic number for negative sentiment news for Associated Press
pd.DataFrame(data[(data["author"] == "Associated Press") & (data["sentiment"] == "negative")]["topic_number_1"].value_counts().sort_values(ascending=False)).reset_index().rename(columns={"index":"topic_number","topic_number_1":"count"}).head(5)

,topic_number,count
0,33,14
1,53,14
2,52,10
3,37,9
4,46,8


* Lets quickly look at the words for the top negative topic

In [235]:
get_topic_word_distribution(33)

{'investigation': '0.0363',
 'attorney': '0.0227',
 'evidence': '0.0196',
 'judge': '0.0185',
 'committee': '0.0178',
 'hearing': '0.0157',
 'justice': '0.0154',
 'letter': '0.0140',
 'document': '0.0138',
 'prison': '0.0134'}

* From the topic words it looks like `crime beat` related news, which makes sense with the assigned `sentiment`

In [225]:
pd.DataFrame(data[(data["author"] == "Associated Press") & (data["sentiment"] == "positive")]["topic_number_1"].value_counts().sort_values(ascending=False)).reset_index().rename(columns={"index":"topic_number","topic_number_1":"count"}).head(5)

,topic_number,count
0,46,19
1,37,9
2,22,8
3,30,7
4,45,7


In [239]:
get_topic_word_distribution(46)

{'voter': '0.0829',
 'poll': '0.0357',
 'vote': '0.0296',
 'ballot': '0.0289',
 'georgia': '0.0255',
 'republican': '0.0184',
 'voting': '0.0164',
 'election': '0.0150',
 'democrats': '0.0127',
 'point': '0.0124'}

* We already looked at topic 46, the articles in this topic are related to `elections` and  `voting`. 

#### Joseph Wulfsohn

* Lets explore another author `Joseph Wulfsohn` and look for patterns and insights there. 

In [198]:
ap_topics = get_topic_count_for_author("Joseph Wulfsohn")
fig = px.pie(ap_topics.head(5), values='count',names='topic_number', title='Topic Distribution For Joseph Wulfsohn')
fig.show()

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



* So the top 3 topics are topic `23`, `2` and `5`. Lets explore these topics.

In [200]:
get_topic_word_distribution_chart(23)

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



* Looks like most of the articles might be related to `social_networks` and `political tweets` which tells me majority of the articles might be tagged `negative` sentiment. 
* Lets take a quick look at the some of the headlines. 

In [244]:
## lets checkout few headlines for topic 23 by Joseph Wulfsohn
data.loc[(data["author"] == "Joseph Wulfsohn") & (data["topic_number_1"] == 23), ["title", "sentiment"]].sample(10).values

array([["He wants to fix social media's 'control' of free speech",
        'positive'],
       ['NYT severs ties with Palestinian freelancer following surfaced social media posts calling to kill Jews',
        'negative'],
       ['Dems trash NY Times over front-page headline deemed too favorable to Trump',
        'negative'],
       ["We aren't trusted because we aren't entirely trustworthy",
        'negative'],
       ["CNN, MSNBC avoid on-air coverage of Bari Weiss' dramatic exit from The New York Times",
        'positive'],
       ["CNN's Jake Tapper retweets criticism of Rebekah Jones after promoting her fake conspiracies on-air",
        'negative'],
       ["The Lincoln Project panned as 'grifters' after report founders looking to launch media empire",
        'positive'],
       ["Hid 'real information' since they might not like the outcome",
        'negative'],
       ["CNN taps new president, current executive producer of 'The Late Show' to replace Jeff Zucker",
        '

* As expected many of the articles seem to be related to various social-media realted activities. It further confirms my intuition that majority of the articles will be tagged `negative` lets check it out.

In [221]:
## sentiment distribution for topic 23 by Joseph Wulfsohn
ap_sentiments = data[(data["topic_number_1"] == 23) & (data["author"] == "Joseph Wulfsohn")]["sentiment"].value_counts().rename_axis('sentiments').reset_index(name='counts')
fig = px.pie(ap_sentiments, values='counts',names='sentiments', title='Sentiment Distribution For Topic 23 By Joseph Wulfsohn', color_discrete_map=color_discrete_map , color='sentiments')
fig.show()

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



* As expected, majority of the articles are tagged `negative` which aligns with my intuition that nowadays `social media` is mostly used to convey negative or aggresive emotions. Which means any news articles covering them tend to be tagged negative. 

## Conclusions

* I cannot confidently say from the given data and analysis if the `Foxnews` is biased towards one party or other. But data does point toward the news source being strongly opinionated. Ideal news sources should be informative and neutral. 
* There were also certain indicators that pointed towards more negative sentiment towars a political party, but I think we need more `data` and need to perform more `Exploratory Data Analysis` to confidently state that. 


## Lessons Learnt

* NLP IS HARD!
* Deep understanding of ML algorithms helps towards understanding the outcomes and performing more effective EDAs.
* Plotting Graphs is more accurate and efficient than eyeballing data during model training. 
* I was more confident for my topic modeling model, after I did a lot of random sampling and manual testing, instead of relying just on performance numbers. 
* EDA - specialy for text data needs a lot of time, I alloted just couple of days to perform EDA on my final data, and I feel there is a lot more to explore. 
* Need to find an efficient way for organizing my `notebooks` after few weeks, I ended up with 10 different notebooks with POCs and scratch-work that made no sense. :( 

## Next Steps

### Technical Improvements

* Create an interactive application, which can make exploration easier and intuitive. 
* Further improvements to `Topic Modeling` algorithms,
    * Fine tuning LDA
    * Exploring advanced `Deep Learning` algorithms like `LDA2Vec` or `BERT`
* Further improvements to `Sentiment Analysis`,
    * Training custom sentiment analysis algorithm based on our dataset. 
    * Exploring `Deep Learning` based algorithms like `BERT`

### Functional Improvements

* Comparing `sentiments` with other news sources, to get more relative idea of the bias. 
* Compare neutrality with `Wikipedia Neutral Point of view (npov) Corpus`